<a href="https://colab.research.google.com/github/Elwing-Chou/ml0930/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
path = "drive/My Drive/news/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall()
path = "drive/My Drive/news/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
import os
import glob
import pandas as pd

def read_data(base):
    contents, targets = [], []
    basedn = os.path.join(base, "*")
    for dn in glob.glob(basedn):
        upper = os.path.join(dn, "*.TXT")
        lower = os.path.join(dn, "*.txt")
        for fn in glob.glob(upper) + glob.glob(lower):
            with open(fn, "r", encoding="utf-8") as f:
                contents.append(f.read())
            target = os.path.split(dn)[-1]
            targets.append(target)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = read_data("chinese_news_trans")
test_df = read_data("chinese_news_test")

In [ ]:
# Series.replace({"李白":0})
# unique/value_counts
cat = train_df["target"].unique()
trans = {w:i for i, w in enumerate(cat)}
trans_r = {i:w for i, w in enumerate(cat)}
y_train = train_df["target"].replace(trans)
y_test = test_df["target"].replace(trans)
y_test

In [ ]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
# apply(函式名字)
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

In [ ]:
p = input("輸入一首新聞:")
x_predict = vec.transform([newscut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(cat, proba):
    print(w, ":", round(p, 4))